# OR 568 - Rogers Rangers
## NYC Rolling Sales Dataset

James Baker

Arturo Davila-Andino

Gridihar Kaushik Ramachandran (GK)

Andrew So

## Developer Notes

*James B*: For tables, I generally build them in excel and then convert them into Markdown tables at these two links:
* https://tableconvert.com/
* https://www.tablesgenerator.com/markdown_tables

# Initial Setup / Package Imports

In [ ]:
# ipak function: install and load multiple R packages.
# check to see if packages are installed. Install them if they are not, then load them into the R session.
ipak <- function(pkg){
    new.pkg <- pkg[!(pkg %in% installed.packages()[, "Package"])]
    if (length(new.pkg)) 
        install.packages(new.pkg, dependencies = TRUE)
    sapply(pkg, require, character.only = TRUE)
}

# usage
packages <- c(
    "ggplot2", 
    "plyr", 
    "mlbench", 
    "e1071", 
    "dplyr", 
    "caret", #  Contains functions to streamline the model training process for complex regression and classification problems
    "pls", # For Partial Least Squares 
    "lars", # For Penalized Models 
    "elasticnet", # For Penalized Models
    "AppliedPredictiveModeling",
    "tidyr",
    "magrittr",
    "stringr",
    "readr",
    "nortest",
    "epiDisplay",
    "corrplot",
    "RColorBrewer", 
    "bestNormalize",
    "randomForest",
    "earth", # For MARS
    "plotmo", # For MARS
    "plotrix", # For MARS
    "TeachingDemos" # For MARS
)
ipak(packages)

Installing packages into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

Warning message:
“dependency ‘R2wd’ is not available”
also installing the dependencies ‘R.oo’, ‘R.utils’, ‘R.cache’, ‘caTools’, ‘lava’, ‘miniUI’, ‘styler’, ‘classInt’, ‘gplots’, ‘matrixStats’, ‘multcomp’, ‘RcppParallel’, ‘fastmatch’, ‘httpuv’, ‘sourcetools’, ‘fastmap’, ‘webshot’, ‘data.table’, ‘gower’, ‘timeDate’, ‘brglm’, ‘gtools’, ‘lme4’, ‘qvcalc’, ‘prodlim’, ‘combinat’, ‘questionr’, ‘ROCR’, ‘mvtnorm’, ‘modeltools’, ‘strucchange’, ‘coin’, ‘zoo’, ‘sandwich’, ‘ISwR’, ‘corpcor’, ‘lamW’, ‘rngtools’, ‘shape’, ‘ParamHelpers’, ‘BBmisc’, ‘checkmate’, ‘parallelMap’, ‘XML’, ‘Deriv’, ‘libcoin’, ‘inum’, ‘MatrixModels’, ‘shiny’, ‘manipulateWidget’, ‘mathjaxr’, ‘sp’, ‘abind’, ‘doParallel’, ‘foreach’, ‘iterators’, ‘itertools’, ‘SparseM’, ‘xtable’, ‘slam’, ‘ModelMetrics’, ‘reshape2’, ‘recipes’, ‘pROC’, ‘BradleyTerry2’, ‘fastICA’, ‘gam’, ‘ipred’, ‘kernlab’, ‘klaR’, ‘ellipse’, ‘mda’, ‘MLmetrics’, ‘party’, ‘proxy’, ‘RAN

## Dataset Import
Dataset found at: https://www.kaggle.com/new-york-city/nyc-property-sales#


We use a a try-catch block in an attempt to find the NYC Rolling Sales data locally; if it doesn't exist, then read it from the hardcoded Google Drive link.

**Note** We used a variety of both manual and automated methods to massage the data before it hits the R pipeline:

* We used a web-facing API called [Geocod.io](https://www.geocod.io/) to add associated latitude/longitudal data for potential use in the model.

* In addition, we used a PowerShell Script `Process-Address.ps1` in an attempt to strip house numbers from the address in an attempt to categorize records by street name (e.g., properties on Broadway, 5th Avenue, 1st Street, etc.)

**These changes resulted in our using a new dataset called `nyc-rolling-sales-latlong-streets.csv`, which the project will attempt to source locally (if it exists) or via an open endpoint on the Team's Google Drive.**

In [ ]:
# The filepath on Google Drive.
drive.fileID <- "1E8NdpOww8IKdN1Kw0NcDPQxycU__hsVu" 
drive.filePath <- sprintf("https://drive.google.com/uc?id=%s",drive.fileID)
file.name <- "nyc-rolling-sales-latlong-streets.csv"

# This is a try-catch block in an attempt to find the NYC Rolling Sales data
# locally; if it doesn't exist, then read it from the hardcoded Google Drive link.
nyc.data <- tryCatch(
    # Try
    {
      message(sprintf("Attempting to read `%s` locally...",file.name))
      read.csv(file.name)
    },
    # Catch
    error = function(cond) {
      message(sprintf("Will use Google Drive Version instead at %s...", drive.filePath))
      return( read.csv(drive.filePath) )
    },
    # Finally
    finally = function(cond) {
      return(message("Unknown error occured!"))
    }
)

cat("Number of Rows: ",nrow(nyc.data), "\nNumber of Cols: ", ncol(nyc.data),"\nShowing first 2 rows...")
head(nyc.data, 2)

## Data Dictionary

https://www1.nyc.gov/assets/finance/downloads/pdf/07pdf/glossary_rsf071607.pdf

|    <br>         	|    <br>Field                                          	|    <br>Description                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                        	|
|-----------------	|-------------------------------------------------------	|-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------	|
|    <br>1        	|    <br>ADDRESS                                        	|    <br>The street   address of the property as listed on the Sales File. Coop sales include the   apartment number in the address field.                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  	|
|    <br>2        	|    <br>APARTMENT NUMBER                               	|    <br>The apartment   number of the property (if applicable)                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             	|
|    <br>3        	|    <br>BLOCK                                          	|    <br>The   block number (up to 5 digits)                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                	|
|    <br>4        	|    <br>BOROUGH                                        	|    <br>A   digit code for the borough the property is located in.<br>   In order, these are:<br>   (1) Manhattan,<br>   (2) Bronx,<br>   (3) Brooklyn,<br>   (4) Queens, and<br>   (5) Staten Island.                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                     	|
|    <br>5/6      	|    <br>BUILDING CLASS (AT PRESENT/ TIME   OF SALE)    	|    <br>The   Building Classification is used to describe a property’s constructive use.   The first position of the Building Class is a letter that is used to describe   a general class of properties (for example “A” signifies one-family homes,   “O” signifies office buildings. “R” signifies condominiums). The second   position, a number, adds more specific information about the property’s use   or construction style (using our previous examples “A0” is a Cape Cod style   one family home, “O4” is a tower type office building and “R5” is a   commercial condominium unit). The term ‘Building Class’ as used by the   Department of Finance is interchangeable with the term Building Code as used   by the Department of Buildings.<br>   <br> <br>   <br>Refer   to code here:<br>   https://www1.nyc.gov/assets/finance/jump/hlpbldgcode.html                                                                                                                                    	|
|    <br>7        	|    <br>BUILDING CLASS CATEGORY                        	|    <br>Refer   to code here:<br>   https://www1.nyc.gov/assets/finance/jump/hlpbldgcode.html                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                              	|
|    <br>8        	|    <br>COMMERCIAL UNITS                               	|    <br>The number   of commercial units at the listed property.                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                           	|
|    <br>9        	|    <br>EASE-MENT                                      	|    <br>An   easement is a right, such as a right of way, which allows an entity to make   limited use of another’s real property. For example: MTA railroad tracks that   run across a portion of another property.                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                       	|
|    <br>10       	|    <br>GROSS SQUARE FEET                              	|    <br>The total   area of all the floors of a building as measured from the exterior surfaces   of the outside walls of the building, including the land area and space   within any building or structure on the property.                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                              	|
|    <br>11       	|    <br>LAND SQUARE FEET                               	|    <br>The land   area of the property listed in square feet.                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             	|
|    <br>12       	|    <br>LOT                                            	|    <br>The   lot number (up to 4 digits)                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  	|
|    <br>13       	|    <br>NEIGHBORHOOD                                   	|    <br>Department   of Finance assessors determine the neighborhood name in the course of valuing   properties. The common name of the neighborhood is generally the same as the   name Finance designates. However, there may be slight differences in   neighborhood boundary lines and some sub-neighborhoods may not be included                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                      	|
|    <br>14       	|    <br>RESIDENTIAL UNITS                              	|    <br>The   number of houses/apartments intended for use as a place of residence at the   address.<br>   (https://www.lawinsider.com/dictionary/residential-unit)                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                        	|
|    <br>15       	|    <br>SALE DATE                                      	|    <br>Date the   property sold.                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                          	|
|    <br>16       	|    <br>SALE PRICE                                     	|    <br>Price paid   for the property.                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                     	|
|    <br>17/18    	|    <br>TAX CLASS (AT PRESENT/TIME OF   SALE)          	|    <br>Property   in NYC is divided into 4 classes:<br>   Class 1: Most residential property of up to three units<br>   (family homes and small stores or offices with one or two apartments   attached),<br>   and most condominiums that are not more than three stories.<br>   <br>   Class 2: All other property that is not in Class 1 and is primarily   residential<br>   (rentals, cooperatives and condominiums). Class 2 includes:<br>   • Sub-Class 2a (4 - 6 unit rental building);<br>   • Sub-Class 2b (7 - 10 unit rental building);<br>   • Sub-Class 2c (2 - 10 unit cooperative or condominium); and<br>   • Class 2 (11 units or more).<br>   <br>   Class 3: Most utility property.<br>   <br>   Class 4: All commercial and industrial properties, such as office, retail,<br>   factory buildings and all other properties not included in tax classes 1, 2   or 3.<br>   <br>   SOURCE:   https://www1.nyc.gov/site/finance/taxes/definitions-of-property-assessment-terms.page    	|

# Data Processing

## Initial Exploration

First, we examine the head of the data to get an idea of the structure.

In [ ]:
# First, we examine the head of the data to get an idea of the structure.
head(nyc.data,2)

# Make all columns lowercase for consistency's sake
colnames(nyc.data) %<>% str_replace_all("\\s", "_") %>% tolower()

To further supplement information gathering, we observe how many predictors there are.

In [ ]:
# Helper function to pretty print column names for a dataframe
prettyPrintCols <- function(data.frame) {
     print(colnames(data.frame) %<>% str_replace_all("\\s", "_") %>% tolower())
}

In [ ]:
# To further supplement information gathering, we observe how many predictors there are.
sprintf("There are %s cols in nyc.data:", ncol(nyc.data))
prettyPrintCols(nyc.data)

Then, we observe the current structure:

In [ ]:
str(nyc.data)

Based on manual observation and what we see in the data here, we observe the following:

* `ease.ment` is 100% null.
* `land.square.feet`, `gross.square.feet`, and `sale.price` are chars when they should be numeric vectors.

So, we will clean the data in the next section.

## Data Cleaning/Transformations in R

### Deep Copy

To begin, we will no longer use `nyc.data` as the dataset for processing in R; we will use a copy to work over and retain the original.

In [ ]:
# Begin with creating a deep copy of the nyc.data Dataframe for use onwards.

df <- nyc.data

#TODO: Be sure to split Y from the data BEFORE we boxcox/preprocess.
# df <- subset(nyc.data, select = -c(sale.price))
# df.y <- nyc.data$sale.price

# Maintain an internal switch to use the STREET and LATLONG predictors.
useStreet <- TRUE
useLatLongs <- FALSE

### Nulling unused predictors

`ease.ment`, `apartment.number`, `sale.date` will not be used in the model because they either 

* Do not fit the purpose of the model, or
* Do not have data that can contribute to the model.

This will also NULL out any other predictors that will not be used in the model.

In [ ]:
# ### Nulling unused predictors
# `ease.ment`, `apartment.number`, `sale.date` will not be used in the model because they either 
# * Do not fit the purpose of the model, or
# * Do not have data that can contribute to the model.
# This will also NULL out any other predictors" that will not be used in the model.

df$ease.ment <- NULL
df$apartment.number <- NULL
df$sale.date <- NULL

# This is a control function to allow usage of the STREET and
# LATLONG predictors within the data set.
if (!useStreet) {
    print("UseStreet is DISABLED.")
    df$street <- NULL
}

if (!useLatLongs) {
    print("UseLATLONGS is DISABLED.")
    df$latitude <- NULL
    df$longitude <- NULL
}
prettyPrintCols(df)

### Recasting predictors

#### Numerics
`sale.price`, `land.square.feet`, and `gross.square.feet` were identified to have been malformed, so they will be re-cast as numeric vectors.

##### Price and Square Footage

In [ ]:
# #### Numerics
# `sale.price`, `land.square.feet`, and `gross.square.feet` were identified to have been malformed, 
# so they will be re-cast as numeric vectors.

##### Price and Square Footage
df$sale.price <- as.numeric(as.character(df$sale.price))
df$land.square.feet <- as.numeric(as.character(df$land.square.feet))
df$gross.square.feet <- as.numeric(as.character(df$gross.square.feet))

#### Factors
The following predictors can be factorized (made into categories):

In [ ]:
# #### Factors
# The following predictors can be factorized (made into categories):
df$borough <- as.factor(df$borough)
df$neighborhood <- as.factor(df$neighborhood)
df$building.class.category <- as.factor(df$building.class.category)

# PRESENT CATEGORIES
df$tax.class.at.present <- as.factor(df$tax.class.at.present)
df$building.class.at.present <- as.factor(df$building.class.at.present)

# TIME OF SALE
df$tax.class.at.time.of.sale <- as.factor(df$tax.class.at.time.of.sale)
df$building.class.at.time.of.sale <- as.factor(df$building.class.at.time.of.sale) 

# ZIP CODE
df$zip.code <- as.factor(df$zip.code)

# Logic to use Street; if so, then remove the Address since it has no 
# utility here.
if (useStreet) {
    df$street <- as.factor(df$street)
    df$address <- NULL
}

#### Factorizing building age

There are a few options that we can do here with respect to the building's age:
We can:
1. Convert to `age` by subtracting from 2017
2. Convert to intervals
3. Convert to factor and leave as is

**The team chooses option 3.**

In [ ]:
#doing option 3
df$year.built <- as.factor(df$year.built)
# df$sale.price = as.numeric(df$sale.price) # ??? This seems redundant

#### Data Filtering

In [ ]:
missRow <- colSums(is.na(df))
plot(missRow)

##### Before Dropping Nulls

We apply a filter to the dataset to do the following:
* Remove observations with incomplete results (lack of dimensions + sale price)

##### Applying filters
The following filters are applied to the dataset:
* Eliminate incomplete observations by dropping records with NULL values in square footage and sale price.
* Reining in gross and land square feet to properties > 100 square feet.
 * This is done to remove records that are either too dirty or have no significant contribution with respect to real estate
* Using only sale prices greater than \\$100,000
 * This is done because price per square foot *realistically* would be (extremely optimistically) $1000/sq. ft in NYC; a minimum

In [ ]:
# drop NA's that were created during the conversion from the "-" value
df <- df %>% drop_na(c(gross.square.feet,land.square.feet, sale.price))

# removing datapoints whose sqftage is less than 100
# removing datapoints whose sale price is <100k. Since median price @nyc is around 1.8k
df <- df %>% filter(land.square.feet>100) 
df <- df %>% filter(gross.square.feet >100)
df <- df %>% filter(sale.price>100000)

In [ ]:
freq_dist_sale_price <- df %>% group_by(sale.price) %>% summarize(Freq = n())
freq_dist_land_square_feet <- df %>% group_by(land.square.feet) %>% summarize(Freq = n())
freq_dist_gross_square_feet <- df %>% group_by(gross.square.feet) %>% summarize(Freq = n())

# TODO: Add additional filters here? As it is, data is heavily skewed towards commercial properties

new_freq_dist_sale_price <- df %>% group_by(sale.price) %>% summarize(Freq = n())
new_freq_dist_land_square_feet <- df %>% group_by(land.square.feet) %>% summarize(Freq = n())
new_freq_dist_gross_square_feet <- df %>% group_by(gross.square.feet) %>% summarize(Freq = n())

In [ ]:
# df <- nyc.data
# df$sale.price <- as.numeric(as.character(df$sale.price))
# df$land.square.feet <- as.numeric(as.character(df$land.square.feet))
# df$gross.square.feet <- as.numeric(as.character(df$gross.square.feet))

# df$ease.ment = NULL
# df$apartment.number = NULL
# df$sale.date = NULL

# df$borough = as.factor(df$borough)
# df$neighborhood = as.factor(df$neighborhood)
# df$building.class.category = as.factor(df$building.class.category)
# df$tax.class.at.present = as.factor(df$tax.class.at.present )

#block and lot

# df$building.class.at.present = as.factor(df$building.class.at.present)

#address needs to be feature engineered
# avenue, broadway etc.

# df$zip.code = as.factor(df$zip.code)

#total units is a calculated field. can be ignored

#convert sqftage to numeric
# df$land.square.feet = as.numeric(df$land.square.feet)
# df$gross.square.feet = as.numeric(df$gross.square.feet)

#need to feature engineer year built- 
# option 1- convert to 'age' by subtracting from 2017
# option 2- convert to intervals
# option 3- convert to factor and leave as is

#doing option 3
# df$year.built <- as.factor(df$year.built)

# df$tax.class.at.time.of.sale = as.factor(df$tax.class.at.time.of.sale)
# df$building.class.at.time.of.sale = as.factor(df$building.class.at.time.of.sale)

# df$sale.price = as.numeric(df$sale.price) 

# removed NA and keep only distinct records
# df <- df %>% distinct() %>% drop_na(c(gross.square.feet,land.square.feet, sale.price))

# #drop NA's that were created during the conversion from the "-" value
# df <- df %>% drop_na(c(gross.square.feet,land.square.feet, sale.price))

# freq_dist_sale_price <- df %>% group_by(sale.price) %>% summarize(Freq = n())
# freq_dist_land_square_feet <- df %>% group_by(land.square.feet) %>% summarize(Freq = n())
# freq_dist_gross_square_feet <- df %>% group_by(gross.square.feet) %>% summarize(Freq = n())

# removing datapoints whose sqftage is less than 100
# removing datapoints whose sale price is <100k. Since median price @nyc is around 1.8k

# df <- df %>% filter(land.square.feet>100) 
# df <- df %>% filter(gross.square.feet >100)
# df <- df %>% filter(sale.price>100000)

# TODO: Add additional filters here? As it is, data is heavily skewed towards commercial properties

# new_freq_dist_sale_price <- df %>% group_by(sale.price) %>% summarize(Freq = n())
# new_freq_dist_land_square_feet <- df %>% group_by(land.square.feet) %>% summarize(Freq = n())
# new_freq_dist_gross_square_feet <- df %>% group_by(gross.square.feet) %>% summarize(Freq = n())

## Descriptive Stats for Price and Square Footage

In [ ]:
# Descriptive Stats
print("For `sale.price`...")
max(df$sale.price)
class(df$sale.price)
max(df$sale.price)
min(df$sale.price)

cat("\n")
print("For `gross.square.feet`...")
max(df$gross.square.feet)
min(df$gross.square.feet)

cat("\n")
print("For `land.square.feet`...")
max(df$land.square.feet)
min(df$land.square.feet)

## Adjusting the data for skewness

To get an idea of the skewness of the data, we will produce a histogram.

We choose `gross.square.feet` arbitrarily.

In [ ]:
# hist(df$sale.price)
hist(df$gross.square.feet)

Based on the histogram above for gross square feet, we observed that the data was heavily skewed and can benefit from a Box Cox + Center/Scaling transformation for improved model performance.

BEFORE we do that though, we will separate `sale.price` from the dataframe to prevent the same transformation from applying to the response.

In [ ]:
# Based on the histogram above for gross square feet, we observed that the data was heavily skewed and can benefit 
# from a Box Cox + Center/Scaling transformation for improved model performance.
# BEFORE we do that though, we will separate `sale.price` from the dataframe to 
# prevent the same transformation from applying to the response.
df.y <- as.data.frame(df$sale.price)
colnames(df.y) <- c("sale.price")

df <- subset(df, select = -c(sale.price))

# Validate that sale.price has been removed from the structure of the main data frame.
prettyPrintCols(df)

### BoxCox / Centering / Scaling

We use the `preProcess()` method to prep a boxcox /centering / scaling routine against the dataset.

In [ ]:
# Preload a preprocessing routine
df.preProc <- preProcess(
    df,
    method = c("BoxCox", "center", "scale")
)

In [ ]:
## Apply the transformations
df.trans <- predict(df.preProc, df)

In [ ]:
hist(df.trans$gross.square.feet)

## Multicollinearity Test

In [ ]:
# # check multicollinearity
# numericpredictors <- c('residential.units','commercial.units','total.units','land.square.feet','gross.square.feet','sale.price')
# df_numpreds <- subset(df, select= numericpredictors)
# corr <- cor(df_numpreds)
# corr
# corrplot(corr,method= "number",col=brewer.pal(n=8, name="PuOr"))

## Data Culling

In [ ]:
# #remove total units as a predictor since it is highly correlated to # of residential units and commercial units

# # one hot encoding the categorical variables
# # borough 
# # neighborhood
# # building.class.category 
# # tax.class.at.present 
# # building.class.at.present
# # zip.code
# # year.built
# # tax.class.at.time.of.sale
# # building.class.at.time.of.sale

# dmy <- dummyVars(" ~ borough +
# neighborhood +
# building.class.category +
# tax.class.at.present +
# building.class.at.present +
# zip.code +
# year.built +
# tax.class.at.time.of.sale +
# building.class.at.time.of.sale", data = df)
# trsf <- data.frame(predict(dmy, newdata = df))
# dim(trsf)

# df_for_modeling <- cbind(trsf, df_numpreds)
# dim(df_for_modeling)

# Modeling

## Partial Least Squares 


### Data Partitioning


In [ ]:
# Set seed
# create a list of random number ranging from 1 to number of rows from actual data 
# and 70% of the data into training data
set.seed(824452)
df.shuffled <- sort(sample(nrow(df.trans), nrow(df.trans)*.7))

# Create both training and test data sets by using the random ordering created 
# in the preceding executed line 
df.pls.train <- as.data.frame(df.trans[df.shuffled,])
df.pls.test <- as.data.frame(df.trans[-df.shuffled,])

df.y.pls.train <- as.data.frame(df.y[df.shuffled,])
df.y.pls.test <- as.data.frame(df.y[-df.shuffled,])

# Rename the columns to `sale.price` since that gets removed in the midst of indexing. 
colnames(df.y.pls.train) <- c("sale.price")
colnames(df.y.pls.test) <- c("sale.price")

## Ordinary Least squares and Robust Linear regression

In [ ]:
df.lm.train <- df.pls.train
df.lm.tree.train <- df.y.pls.train
df.lm.test <- df.pls.test

linearmodel = lm(df.lm.train$sale.price ~ .,df.lm.train)
lm_prediction <- predict(linearmodel,df.lm.test)

In [ ]:
#Robust linear regression
robust_linearmodel = rlm(df.lm.train$sale.price ~ .,df.lm.train)
rlm_prediction = predict(robust_linearmodel,df.lm.test)


### Create control for PLS Model

This is the `trainControl` method used for Partial Least Squares; this model will use 10-folds Cross Validation for model resampling.


In [ ]:
# create control
pls.ctrl <- trainControl(method = "cv", number = 10)

### Partial Least Squares

In [ ]:
str(df.pls.train)

#### Manual PLS

We set the number of components to measure against as 10, then we will use the 
`plsr()` function to invoke the partial least squares regression.

In [ ]:
numComponents <- 10
plsFit.manual = plsr(df.y.pls.train$sale.price ~., data = df.pls.train, ncomp = numComponents)

In [ ]:
summary(plsFit.manual)

In [ ]:
# try on the test data, and then evaluate the results.
plsPred.manual = predict(plsFit.manual, df.pls.test, ncomp = numComponents)
plsValue.manual = data.frame(obs = df.y.pls.test$sale.price, pred = plsPred.manual[,,1])

In [ ]:
cat("Partial Least Squares (Manual)")
defaultSummary(plsValue.manual)
cat("\n")

Based on the results of the manual PLS model, we can see that $R^{2}\,\approx\,0.17$, which indicates that the model was ill-suited to predict or observe a linear relationship between all of the predictors.

#### `TrainControl`-based PLS with 10-folds cross validation

To verify/validate the results seen in the manual PLS model, we will use a `TrainControl` to automate a 10-fold cross validation of a PLS model. 

In [ ]:
# #### `TrainControl`-based PLS with 10-folds cross validation
# To verify/validate the results seen in the manual PLS model, we will use a `TrainControl` 
# to automate a 10-fold cross validation of a PLS model. 

# Partial Least Squares - Automated
plsFit.CV <- train(
    x = df.pls.train, # select features/predictors
    y = df.y.pls.train$sale.price, # select outcome
    method = "pls",
    tuneGrid = expand.grid(ncomp = numComponents),
    trControl = pls.ctrl
)

In [ ]:
summary(plsFit.CV)

In [ ]:
plsPred.CV <- predict(plsFit.CV, df.pls.test)

In [ ]:
plsTest.CV <- data.frame(obs = df.y.pls.test$sale.price, pred = predict(plsFit.CV, df.pls.test))
defaultSummary(plsTest.CV)

Based on the results seen above, the results are virtually indistinguishable from the manual PLS model.

## Tree Based Regression

We will use the same partitioned data from the Partial Least Squares Section.

In [ ]:
df.tree.train <- df.pls.train
df.y.tree.train <- df.y.pls.train

In [ ]:
# Setting the seed for our Regression Tree Model
set.seed(4000)

treebasedfit <- train(
  x= df.tree.train, 
  y= df.y.tree.train$sale.price,
  method = "rpart2",
  tuneLength = 20,
  trControl = trainControl(method = "cv")
)

treebasedfit
plot(treebasedfit)

## Elastic Net

In [ ]:
# GK's Code to convert to binary predictors
numericpredictors <- c('residential.units','commercial.units','total.units','land.square.feet','gross.square.feet')
df_numpreds <- subset(df, select= numericpredictors)

dmy <- dummyVars(" ~ borough +
neighborhood +
building.class.category +
tax.class.at.present +
building.class.at.present +
zip.code +
year.built +
tax.class.at.time.of.sale +
building.class.at.time.of.sale", data = df)
trsf <- data.frame(predict(dmy, newdata = df))
dim(trsf)

df_for_modeling <- cbind(trsf, df_numpreds)
dim(df_for_modeling)

# Partition Data
nearZeroVar(df_for_modeling)
df.enet.train <- df_for_modeling[df.shuffled,-nearZeroVar(df_for_modeling)]
df.enet.test <- df_for_modeling[-df.shuffled,-nearZeroVar(df_for_modeling)]

df.y.enet.train <- df.y[df.shuffled,]
df.y.enet.test <- df.y[-df.shuffled,]

# Train Control
enet.ctrl <- trainControl("LGOCV")

# Tune Grid
k=10


# Train Model
enetFit.MC <- train(x = df.enet.train, # select features/predictors
                   y = df.y.enet.train, # select outcome
                   method = "enet",
                   tuneGrid = expand.grid(.fraction = .2*(1:k)/k, .lambda=.001*(1:k)/k),
                   trControl = enet.ctrl,
                   preProc = c("center", "scale"))

# Summary
summary(enetFit.MC)
enetFit.MC$bestTune
plot(enetFit.MC)

#Preditction
enetPred.MC <- predict(enetFit.MC, df.enet.test)

enetTest.MC <- data.frame(obs = df.y.enet.test, pred = predict(enetFit.MC, df.enet.test))
defaultSummary(enetTest.MC)


## MARS

In [ ]:
# Partition Data
df.mars.train <- df.pls.train
df.mars.test <- df.pls.test

df.y.mars.train <- df.y[df.shuffled,]
df.y.mars.test <- df.y[-df.shuffled,]

# Train Control
mars.ctrl <- trainControl("LGOCV")

# Tune Grid
marsGrid = expand.grid(.degree=1:3, .nprune=2:15)

# Train Model Packages must be loaded prior 
marsFit.MC <- train(x = df.mars.test, # select features/predictors
                   y = df.y.mars.test, # select outcome
                   method = "earth",
                   tuneGrid = marsGrid,
                   trControl = mars.ctrl)

warnings()

# Summary
summary(marsFit.MC)
marsFit.MC$bestTune
plot(marsFit.MC)

#Prediction
marsPred.MC <- predict(marsFit.MC, df.mars.test)

marsTest.MC <- data.frame(obs = df.y.mars.test, pred = predict(marsFit.MC, df.mars.test))
defaultSummary(marsTest.MC)

# RAW CODE - GK

In [ ]:
# head(nycdata)

# colnames(nycdata) %<>% str_replace_all("\\s", "_") %>% tolower()
# colnames(nycdata)

# str(nycdata)

# df <- nycdata
# df$sale.price <- as.numeric(as.character(df$sale.price))
# df$land.square.feet <- as.numeric(as.character(df$land.square.feet))
# df$gross.square.feet <- as.numeric(as.character(df$gross.square.feet))

# df$ease.ment = NULL
# df$apartment.number = NULL
# df$sale.date = NULL

# df$borough = as.factor(df$borough)
# df$neighborhood = as.factor(df$neighborhood)
# df$building.class.category = as.factor(df$building.class.category)
# df$tax.class.at.present = as.factor(df$tax.class.at.present )

# #block and lot

# df$building.class.at.present = as.factor(df$building.class.at.present)

# #address needs to be feature engineered
# # avenue, broadway etc.

# df$zip.code = as.factor(df$zip.code)

# #total units is a calculated field. can be ignored

# #convert sqftage to numeric
# df$land.square.feet = as.numeric(df$land.square.feet)
# df$gross.square.feet = as.numeric(df$gross.square.feet)

# #need to feature engineer year built- 
# # option 1- convert to 'age' by subtracting from 2017
# # option 2- convert to intervals
# # option 3- convert to factor and leave as is

# #doing option 3
# df$year.built <- as.factor(df$year.built)

# df$tax.class.at.time.of.sale = as.factor(df$tax.class.at.time.of.sale)
# df$building.class.at.time.of.sale = as.factor(df$building.class.at.time.of.sale)

# df$sale.price = as.numeric(df$sale.price)

# # removed NA and keep only distinct records
# df <- df %>% distinct() %>% drop_na(c(gross.square.feet,land.square.feet, sale.price))

# #drop NA's that were created during the conversion from the "-" value
# df <- df %>% drop_na(c(gross.square.feet,land.square.feet, sale.price))

# freq_dist_sale_price <- df %>% group_by(sale.price) %>% summarize(Freq = n())
# freq_dist_land_square_feet <- df %>% group_by(land.square.feet) %>% summarize(Freq = n())
# freq_dist_gross_square_feet <- df %>% group_by(gross.square.feet) %>% summarize(Freq = n())

# freq_dist_sale_price
# freq_dist_land_square_feet
# freq_dist_gross_square_feet

# # removing datapoints whose sqftage is less than 100
# # removing datapoints whose sale price is <100k. Since median price @nyc is around 1.8k

# df <-df%>% filter(land.square.feet>100) 
# df<- df %>% filter(gross.square.feet >100)
# df<- df %>% filter(sale.price>100000)

# new_freq_dist_sale_price <- df %>% group_by(sale.price) %>% summarize(Freq = n())
# new_freq_dist_land_square_feet <- df %>% group_by(land.square.feet) %>% summarize(Freq = n())
# new_freq_dist_gross_square_feet <- df %>% group_by(gross.square.feet) %>% summarize(Freq = n())

# max(nycdata$sale.price)
# class(nycdata$sale.price)
# max(df$sale.price)
# min(df$sale.price)

# max(df$gross.square.feet)
# min(df$gross.square.feet)

# max(df$land.square.feet)
# min(df$land.square.feet)

# hist(df$sale.price)
# hist(df$gross.square.feet)

# # check multicollinearity
# install.packages("corrplot")
# library(corrplot)
# library(RColorBrewer)
# numericpredictors <- c('residential.units','commercial.units','total.units','land.square.feet','gross.square.feet','sale.price')
# df_numpreds <- subset(df, select= numericpredictors)
# corr <- cor(df_numpreds)
# corr
# corrplot(corr,method= "number",col=brewer.pal(n=8, name="PuOr"))

# #remove total units as a predictor since it is highly correlated to # of residential units and commercial units

# # one hot encoding the categorical variables
# # borough 
# # neighborhood
# # building.class.category 
# # tax.class.at.present 
# # building.class.at.present
# # zip.code
# # year.built
# # tax.class.at.time.of.sale
# # building.class.at.time.of.sale

# dmy <- dummyVars(" ~ borough +
# neighborhood +
# building.class.category +
# tax.class.at.present +
# building.class.at.present +
# zip.code +
# year.built +
# tax.class.at.time.of.sale +
# building.class.at.time.of.sale", data = df)
# trsf <- data.frame(predict(dmy, newdata = df))
# dim(trsf)

# df_for_modeling <- cbind(trsf, df_numpreds)
# dim(df_for_modeling)

# #split data into train, validation and test
# inTraining <- createDataPartition(df_for_modeling$sale.price, p = .70, list = FALSE)
# trainset <- df_for_modeling[ inTraining,]
# testandvalidation  <- df_for_modeling[-inTraining,]

# testid <- createDataPartition(testandvalidation$sale.price, p = .50, list = FALSE)
# testset <- testandvalidation[testid,]
# validationset <- testandvalidation[-testid,]

# dim(trainset)
# dim(validationset)
# dim(testset)

# # training various models

# #Simple Linear Regression

In [ ]:
plot(nycdata[2],main = colnames(nycdata[2]))
plot(nycdata[3],main = colnames(nycdata[3]))
plot(nycdata[4],main = colnames(nycdata[4]))
plot(nycdata[5],main = colnames(nycdata[5]))
plot(nycdata[6],main = colnames(nycdata[6]))
plot(nycdata[7],main = colnames(nycdata[7]))
plot(nycdata[8],main = colnames(nycdata[8]))
#plot(nyc.data[9],main = colnames(nyc.data[9])) Address not done
# library(hexbin)
#hexbinplot(latitude~longitude, data=nycdata)
#plot(nyc.data[10],main = colnames(nyc.data[10]))
#plot(nyc.data[11],main = colnames(nyc.data[11]))
#plot(nyc.data[12],main = colnames(nyc.data[12])) Apt number not done
plot(nyc.data[13],main = colnames(nycdata[13]))
histogram(nycdata$residential.units)
histogram(nycdata$commercial.units)
histogram(nycdata$total.units)
histogram(nycdata$land.square.feet)
histogram(nycdata$gross.square.feet)
histogram(nycdata$year.built)
plot(nyc.data[20],main = colnames(nycdata[20]))
plot(nyc.data[21],main = colnames(nycdata[21]))
histogram(nycdata$sale.price)

In [ ]:
nyc.data2 <- nyc.data
nyc.data2 <- nyc.data2[which(nyc.data$LATITUDE <= 41),]
nyc.data2 <- nyc.data2[which(nyc.data2$LATITUDE >= 40),]
nyc.data2 <- nyc.data2[which(nyc.data2$LONGITUDE <= -73.6),]
hexbinplot(LATITUDE~LONGITUDE, data=nyc.data2)
nyc.data2 <- nyc.data2[which(nyc.data2$LAND.SQUARE.FEET <= 1000000),]
nyc.data2 <- nyc.data2[which(nyc.data2$LAND.SQUARE.FEET >= 100),]
nyc.data2 <- nyc.data2[which(nyc.data2$GROSS.SQUARE.FEET >= 100),]
nyc.data2 <- nyc.data2[which(nyc.data2$SALE.PRICE >= 100000),]
nyc.data2 <- nyc.data2[which(nyc.data2$YEAR.BUILT >= 1500),]